In [4]:
import numpy as np
from torchvision.datasets import MNIST
def download_mnist(is_train: bool):
    dataset = MNIST(root='./data',
            transform=lambda x: np.array(x).flatten(),
            download=True,
            train=is_train)
    mnist_data = []
    mnist_labels = []
    for image, label in dataset:
        mnist_data.append(image)
        mnist_labels.append(label)
    return mnist_data, mnist_labels
train_X, train_Y = download_mnist(True)
test_X, test_Y = download_mnist(False)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9912422/9912422 [00:00<00:00, 181274683.65it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28881/28881 [00:00<00:00, 25839525.13it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1648877/1648877 [00:00<00:00, 88533608.95it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4542/4542 [00:00<00:00, 4334591.30it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Normalize the data and convert the labels to one-hot-encoding

In [5]:
train_X = np.array(train_X) / 255.0
test_X = np.array(test_X) / 255.0

In [6]:
def one_hot_encoding(labels):
  # labels = labels.astype(int)
  one_hot_encode = np.zeros((len(labels), 10))
  for i in range (len(labels)):
    one_hot_encode[i, labels[i]] = 1
  return one_hot_encode

train_Y = one_hot_encoding(train_Y)
test_Y = one_hot_encoding(test_Y)

In [7]:
def init_param(input, output):
  W = np.random.rand(input, output)
  b = np.zeros((output,))
  return W, b

#print(init_param(784, 10))


Softmax, Cross Entrpy Loss, Gradient Descent

In [8]:
def softmax(x):
  exp_x = np.exp(x - np.max(x, axis = 1, keepdims = True))
  return exp_x / np.sum(exp_x, axis = 1, keepdims = True)

In [9]:
def cross_entropy(true_y, prediction_y):
  m = true_y.shape[0]
  log_p = -np.log(prediction_y[range(m), true_y.argmax(axis=1)])
  loss = np.sum(log_p) / m

  return loss


In [10]:
def gradient_descent(X, true_y, prediction_y, W, b, learning_rate):
  error = true_y - prediction_y
  W += learning_rate * (np.dot(X.T, error))
  b += learning_rate * np.sum(error, axis = 0)
  return W, b

Prediction, Accuracy

In [11]:
def predict(X, W, b):
  z = np.dot(X, W) + b
  prediction_y = softmax(z)
  #return prediction_y
  return np.argmax(prediction_y, axis = 1)

In [12]:
def accuracy(X, Y, W, b):
  prediction_y = predict(X, W, b)
  true_y = np.argmax(Y, axis = 1)

  acc = np.mean(prediction_y == true_y)
  return acc

Training

In [13]:
def train(train_X, train_Y, W, b, epochs=100, batch_size=100, learning_rate=0.01):

  for epoch in range(epochs):
    i = np.random.permutation(train_X.shape[0])
    train_X_random = train_X[i]
    train_Y_random = train_Y[i]

    for i in range(0, train_X.shape[0], batch_size):
      batch_X = train_X_random[i : i + batch_size]
      batch_Y = train_Y_random[i : i + batch_size]

      #prediction_y = predict(batch_X, W, b)
      z = np.dot(batch_X, W) + b
      prediction_y = softmax(z)

      loss = cross_entropy(batch_Y, prediction_y)

      W, b = gradient_descent(batch_X, batch_Y, prediction_y, W, b, learning_rate)

    print(f"Epoch: {epoch+1} Loss: {loss}")

  return W, b


In [14]:
W, b = init_param(784,10)
trained_W, trained_b = train(train_X, train_Y, W, b, 100, 100, 0.01)

test_acc = accuracy(test_X, test_Y, trained_W, trained_b)
print(f"Test accuracy: {test_acc * 100:.2f}% ")

Epoch: 1 Loss: 0.569490353873796
Epoch: 2 Loss: 0.21076225738622154
Epoch: 3 Loss: 0.26924263306954244
Epoch: 4 Loss: 0.1368058070604183
Epoch: 5 Loss: 0.24766449538141724
Epoch: 6 Loss: 0.3486496826360361
Epoch: 7 Loss: 0.22865669189248664
Epoch: 8 Loss: 0.348208976895903
Epoch: 9 Loss: 0.21967654637165104
Epoch: 10 Loss: 0.4757673895595647
Epoch: 11 Loss: 0.2455511220487835
Epoch: 12 Loss: 0.3962336028256068
Epoch: 13 Loss: 0.2469593089874096
Epoch: 14 Loss: 0.25239733145827126
Epoch: 15 Loss: 0.27042291359455656
Epoch: 16 Loss: 0.38059736472938427
Epoch: 17 Loss: 0.4438110055504936
Epoch: 18 Loss: 0.21105555183170627
Epoch: 19 Loss: 0.17278968951649248
Epoch: 20 Loss: 0.14361617477271277
Epoch: 21 Loss: 0.32285933027794195
Epoch: 22 Loss: 0.20837129935853888
Epoch: 23 Loss: 0.23148352612397965
Epoch: 24 Loss: 0.1771309348197768
Epoch: 25 Loss: 0.23119851610319406
Epoch: 26 Loss: 0.3559026437851414
Epoch: 27 Loss: 0.21313681629107994
Epoch: 28 Loss: 0.2750951610993228
Epoch: 29 Loss: